# 네이버 블로그 크롤링 (네이버 API 이용)

## 들어가며...



## 1) 크롤링? 파싱?

### 1-1) 크롤링

- '웹 크롤러'라는 단어에서 유래되었음.
- 크롤러란 조직적, 자동화된 방법으로 WWW를 탐색하는 컴퓨터 프로그램.
- 크롤링은 크롤러가 하는 작업을 부르는 말.
- 여러 인터넷 사이트의 페이지(문서,html 등)를 수집해서 분류하는 것.
- 대체로 찾아낸 데이터를 저장한 후 쉽게 찾을 수 있게 인덱싱 수행.

### 1-2) 파싱

- 파싱이란 어떤 페이지(문서, html 등)에서 내가 원하는 데이터를 특정 패턴이나 순서로 추출하여 정보를 가공하는 것.
- 파싱이란 일련의 문자열을 의미있는 '토큰'으로 분해하고 이들로 이루어진 '파스 트리'를 만드는 과정.
- 입력 토큰에 내제된 자료 구조를 빌드하고 문법을 검사하는 역할을 함.

## 2) 애플리케이션 등록 ( API 이용신청)

![애플리케이션 등록_1](https://user-images.githubusercontent.com/51112316/61574953-2f655d80-ab01-11e9-8b8f-cc74183302da.jpg)
![애플리케이션 등록_2](https://user-images.githubusercontent.com/51112316/61574954-31c7b780-ab01-11e9-9274-e1d237547d94.jpg)

### 2-1) 네이버API  예제 사용

- 네이버개발자 -> Products -> 서비스API -> 검색 -> 개발 가이드 보기 -> 0.API호출예제 -> Python

#### [API호출예제](https://developers.naver.com/docs/search/blog/)

~~~python
import java.io.BufferedReader;
import java.io.InputStreamReader;
import java.net.HttpURLConnection;
import java.net.URL;
import java.net.URLEncoder;

public class APIExamSearchBlog {

    public static void main(String[] args) {
        String clientId = "YOUR_CLIENT_ID";//애플리케이션 클라이언트 아이디값";
        String clientSecret = "YOUR_CLIENT_SECRET";//애플리케이션 클라이언트 시크릿값";
        try {
            String text = URLEncoder.encode("그린팩토리", "UTF-8");
            String apiURL = "https://openapi.naver.com/v1/search/blog?query="+ text; // json 결과
            //String apiURL = "https://openapi.naver.com/v1/search/blog.xml?query="+ text; // xml 결과
            URL url = new URL(apiURL);
            HttpURLConnection con = (HttpURLConnection)url.openConnection();
            con.setRequestMethod("GET");
            con.setRequestProperty("X-Naver-Client-Id", clientId);
            con.setRequestProperty("X-Naver-Client-Secret", clientSecret);
            int responseCode = con.getResponseCode();
            BufferedReader br;
            if(responseCode==200) { // 정상 호출
                br = new BufferedReader(new InputStreamReader(con.getInputStream()));
            } else {  // 에러 발생
                br = new BufferedReader(new InputStreamReader(con.getErrorStream()));
            }
            String inputLine;
            StringBuffer response = new StringBuffer();
            while ((inputLine = br.readLine()) != null) {
                response.append(inputLine);
            }
            br.close();
            System.out.println(response.toString());
        } catch (Exception e) {
            System.out.println(e);
        }
    }
}
~~~

#### 문제 발생 :  

#### 네이버 블로그 형식 맞춰 긁어왔음.
                  
#### 사실은 요약된 내용.

#### 모든 내용을 다 보고싶으면 개발이 필요.

# 네이버 블로그 크롤링

In [18]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup

naver_Client_id = "E4zDnlSmVAnJ7dvIOUUx"
naver_Client_secret = "lLxEw7DWbB"

In [24]:
def get_blog_count(query,display) :
    encode_query=urllib.parse.quote(query)
    search_url="https://openapi.naver.com/v1/search/blog?query="+ encode_query
    # 위의 https://~ 는 따온 API호출예제에 있음.
    request=urllib.request.Request(search_url)
    # 이렇게 하면 내가 만들었던 url 형태로 요청한다.
    # 그러나 이렇게 하면 API로부터 거부당한다.
    # 왜냐하면 ID, Secret을 주지 않았기 때문에 아래와 같이 이것을 입력해준다.
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    # code를 받아와서 웹에서 정상적으로 접속했을 때 200번을 준다.
    if response_code is 200 :
        response_body = response.read()  # 실제로 읽어 들인다.
        response_body_dict = json.loads(response_body.decode('utf-8'))
        # 네이버에서 넘겨줄 때 json으로 넘겨주는데 이것을 전부 '딕셔너리'로 저장한다.
        
        print('Last build date :' + str(response_body_dict['lastBuildDate']))
        # json으로 넘겨받을 것들 중에서 최종날짜를 출력
        print('Total :' + str(response_body_dict['total']))  # 관측개수
        print('Start :' + str(response_body_dict['start']))
        print('Display :' + str(response_body_dict['display']))
        #이렇게 넘어온 것을 다 출력해본다.
        
        # 검색된 결과가 0 일 경우        
        if response_body_dict['total']== 0:
            blog_count = 0
        else :
            blog_total=math.ceil(response_body_dict['total']/int(display))
            # 위에서 한 번에 10개씩 출력한다고 했으니 나눠주어서 개수를 맞춰준다.
        
            if blog_total >= 1000 :
                blog_count = 1000
                # 최대 1000개로 제한이 있으니 1000개를 넘었을 때는 1000개로 지정한다.
            else : 
                blog_count = blog_total
        
            print('Blog total :' + str(blog_total))
            print('Blog count : ' + str(blog_count))
        
        return blog_count

- 블로그의 모든 내용을 가져오는 함수[(API사용예제)는 샘플 내용만 가져온다.]



In [38]:
def get_blog_post(query,display,start_index,sort):
    global no, fs
    #get_blog_count(query,display)를 정의할 때 사용했던 것과 동일하게 사용(아래 7문장)
    encode_query = urllib.parse.quote(query)
    # 동일하게 사용하나 search_url에는 옵션을 조금 더 추가해 준다.
    search_url = "https://openapi.naver.com/v1/search/blog?query=" + encode_query +"&display=" + str(display) + "&start=" + str(start_index) + "sort=" + sort
    request=urllib.request.Request(search_url)
    
    request.add_header("X-Naver-Client-Id",naver_Client_id)
    request.add_header("X-Naver-Client-Secret",naver_Client_secret)
    
    response=urllib.request.urlopen(request)
    response_code=response.getcode()
    
    #print(response_code) 이분은 윗부분까지 해서 접속이 잘되었는지를 확인하는 것.
        
    if response_code is 200:
        response_body=response.read()
        response_body_dict=json.loads(response_body.decode('utf-8'))
        # json이 해당하는 블로그가 여러개 나오면 전부 다 하나의 큰 덩어리로 주는데
        # 나눠서 하나씩 접근 하기 위해 아래의 코드를 짜줌.
        for item_index in range(0,len(response_body_dict['items'])):
            # 혹시 태그부분이 따라올 수 있기 때문에
            # 그것을 지우고 사용하기 위해  re(regular expression)으로 제거를 해준다.
            # 즉 태그를 없애주는 정규표현식을 이용한다.
            try :
                remove_html_tag = re.compile('<.*?>')
                title = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['title'])
                #또한 주소가 따라오는데 이건 필요 없는 것이다.그러므로 없애준다.
                link = response_body_dict['items'][item_index]['link'].replace("amp;","")
                # description에도 마찬가지로 태그가 따라올 수 있기 때문에 제거해준다.
                description = re.sub(remove_html_tag,'',response_body_dict['items'][item_index]['description'])
                blogger_name = response_body_dict['items'][item_index]['bloggername']
                #  포스트(개시글)의 주소가 아니라 블로그 자체의 링크를 가져옴.
                blogger_link = response_body_dict['items'][item_index]['bloggerlink']
                post_date = response_body_dict['items'][item_index]['postdate']
                #아래처럼 변환해서 가져와도 되고 그냥 주는대로 가져와도 된다.
                #post_date = datetime.datetime.strptime(response_body_dict[['item'][item_index]['postdate'],"%Y%m%d").strtime("%Y.%m.%d")
                
               # 이제 json에서 오는 결과를 하나씩 items에 다 들어가 있다. 그것을 이제 다 끄집어서 가져온다.
                no +=1
                print("-------------------------------------------------") #각각의 게시글을 구분시킴
                print("#"+ str(no))  #몇번째 게시글인지 출력
                print("Title :"+ title)
                print("Link :"+ link)
                print("Description :"+ description)
                print("Blogger Name :"+ blogger_name)
                print("Blogger Link :"+ blogger_link)
                print("Post Date :"+ post_date)
                
                                                                              
                post_code = request.get(link)
                #  저 정보들은 사실 네이버 오픈 API에서 제공해주는 정보다.
                # 그런데 우리가 필요한 것은 실제 블로그에 포스트 된 내용이 필요하기 때문에
                # 뷰티풀숲을 이용해 가져온다. 
                post_text = post_code.text
                post_soup = BeautifulSoup(post_text,'lxml')
                
                # 네이버 블로그의 문제점 : 마우스를 긁어오지 못하게 iframe으로 만들어 놓은게 있다.
                #그래서 iframe#mainFrame 이라는 html 부분을 들어가서
                # 그 부분의 실제 post url을 뽑을 수 있다.
                # 그냥 가져와선 안되고 그 url에서 소스부분(src)만 가져온다.
                # 그래서 그걸로 접속해서 실제적인 블로그 포스트에 대한 내용을 긁어오기 때문에
                # 여기에 다시 한번 "blog_post_url"이라는 주소로 접근을 한다.
                # 그리고 그것들에 대한 정보를 가져온다.
                for mainFrame in post_soup.select('iframe#mainFrame'):
                    blog_post_url = "http://blog.naver.com"+mainFrame.get('src')
                    blog_post_code = requests.get(blog_post_url)
                    blog_post_text = blog_post_code.text
                    blog_post_soup = BeautifulSoup(blog_post_text,'lxml')
                    
                    # 블로그 전체의 여러가지 메뉴들도 있고 그런 것 제외하고 postViewArea라는 부분에
                    # 실제 포스트 내용이 들어있다. 그래서 여기로 선택한다. 이게 진짜 블로그 포스트의 컨텐트다. 
                    for blog_post_content in blog_post_soup.select("div#postViewArea"):
                        blog_post_content_text = blog_post_content.get_text()
                        blog_post_full_contents = str(blog_post_content_text) #컨텐트를 string으로 캐스팅해서 가져온다.
                        blog_post_full_contents = blog_post_full_contents.replace("\n\n",'\n') 
                        #요약된 description 만 주는게 아니라 이젠 진짜로 된 포스트를 준다.
                        #print("blog_post_contents : " + blog_post_full_contents+"\n")
                        #전체 내용을 print 하면 너무 많기 때문에 위의 코드로 하지않고 파일로 저장한다.
                        #이를 통해 이후에 데이터 분석 등에 사용한다.
                        fs.write(blog_post_full_contents+'\n')
                        fs.write("-----------------------------------\n")
            #만약에 못할 경우 그냥 넘어가는 코드.
            except: 
                item_index+=1

                                                                              
if __name__=='__main__' :
    no = 0           # 몇 개의 포스트를 저장했는지 세기 위한 index
    query = "문재인" # 검색을 원하는 문자열로써 UTF-8로 인코딩한다.
    display = 10     # 검색 결과 출력 건수 지정, 10(기본값), 100(최대값)
    start = 1        # 검색 시작 위치로 최대 1000까지 가능
    sort = 'date'    # 정렬 옵션 : sim(유사도순, 기본값), date(날짜순)
    
    #블로그 콘텐츠의 한글 저장을 위해 encoding='utf-8'으로 설정.
    fs = open(query + ".txt",'a',encoding='utf-8')
    
    blog_count = get_blog_count(query,display)
    for start_index in range(start,blog_count + 1, display):
        get_blog_post(query,display,start_index,sort)
        
    fs.close()

Last build date :Sat, 20 Jul 2019 17:58:31 +0900
Total :1445584
Start :1
Display :10
Blog total :144559
Blog count : 1000
-------------------------------------------------
#1
Title :&quot;대형병원 환자 쏠림, 문재인 케어 탓 아냐…
Link :https://blog.naver.com/shyu3?Redirect=Log&logNo=221590531153
Description :&quot;대형병원 환자 쏠림, 문재인 케어 탓 아냐…지난 10년간 상급종합병원 진료비 점유율 외래 5%, 입원 20... [메디게이트뉴스 정다연 기자] 빅5병원 환자쏠림에 대한 문제제기가 다시 수면 위로 떠오르면서 문재인... 
Blogger Name :유석희 교수의 '慶山亭'
Blogger Link :https://blog.naver.com/shyu3
Post Date :20190720
-------------------------------------------------
#2
Title :&quot;문재인 헌법 위반 못 보겠다&quot;...변호사 37명 '집단청원'
Link :https://blog.naver.com/kst3175?Redirect=Log&logNo=221589028863
Description :&quot;문재인 헌법 위반 못 보겠다&quot;… 변호사 37명 '집단청원'한변 등 변호사 37명 &quot;대통령이 자유민주... ⓒ이종현 기자 우파 진영 변호사 37명이 “헌법 위반 행위를 시정하라”며 17일 문재인 대통령에게 청원서를... 
Blogger Name :지구촌다문화 뉴스
Blogger Link :https://blog.naver.com/kst3175
Post Date :20190718
-------------------------------------------------
#3
Title :김능환의 번복 + 문재

-------------------------------------------------
#31
Title :문재인과 북한 외무성 일개 국장 중 누구 말이 거짓이고... 
Link :https://blog.naver.com/choisttt?Redirect=Log&logNo=221573779754
Description :문재인은 얼마전 7개국 통신사와 서면 인터뷰를 했습니다. 서면 인터뷰는 직접 인터뷰보다 현실감은... 그런데 말이죠, 문재인의 통신사 인터뷰가 국내 신문에 보도된 그날 아침 권정근 북한 외무성 미국담당... 
Blogger Name :달리는 사나이 최석태
Blogger Link :https://blog.naver.com/choisttt
Post Date :20190629
-------------------------------------------------
#32
Title :변호사 문재인과 대통령 문재인       -배명복(중앙일보 대기자)-
Link :https://blog.naver.com/edu9508?Redirect=Log&logNo=221586759618
Description :“문재인 정권은 허구와 위선의 역사의식에서 벗어나기는커녕 조선조 통치체제의 정신적 퇴물인... 있다.” 문재인 정부에 대한 그의 비판이다. 양 교수의 글을 길게 인용한 것은 그 어느 때보다 균형 잡힌... 
Blogger Name :edu9508(국어공부) 도우미
Blogger Link :https://blog.naver.com/edu9508
Post Date :20190716
-------------------------------------------------
#33
Title :일본 후지TV 논설위원 '문재인 탄핵이 해법' 막말 논란
Link :https://blog.naver.com/rehouse?Redirect=Log&logNo=221589873179
Description :일본 후지TV 논설위원 “문재인 탄핵이 해법” 막말 논란 출처 : 서울신문 

#66
Title :문재인 대통령과 여야 5당 대표 공동발표문
Link :https://blog.naver.com/thebluehousekr?Redirect=Log&logNo=221589581471
Description :문재인 대통령과 여야 5당 대표는 일본의 수출 규제 조치에 대해 심도 있게 논의하고 공동발표문을... &lt;공동발표문&gt; ○ 고민정 청와대 대변인 : 문재인 대통령과 여야 5당 대표는 일본의 수출 규제 조치에 대하여... 
Blogger Name :대한민국 청와대 공식 블로그입니다.
Blogger Link :https://blog.naver.com/thebluehousekr
Post Date :20190719
-------------------------------------------------
#67
Title :일본의 경제 제재와 문재인
Link :https://blog.naver.com/blueload487?Redirect=Log&logNo=221590045287
Description :-일본의 경제 제재와 문재인 - 나는 일본의 오만과 끊임없는 괴롭힘에 대해 지극히 분노하는 사람이다.... 그런 내가 이번 일본의 경제제재에 대해서, 일본보다는 문재인에게 할 말이 많다. 마치 실력도 없는 놈이... 
Blogger Name :blueload487님의 블로그
Blogger Link :https://blog.naver.com/blueload487
Post Date :20190719
-------------------------------------------------
#68
Title :문재인케어 10대 문제점
Link :https://blog.naver.com/saenuriparty?Redirect=Log&logNo=221577611500
Description :문재인케어 10대 문제점 ◆ 문재인케어 10대 문제점 &quot;건보료 폭탄! 자유한국당이 막아내겠습니다!&quot; 1. 향후 5년간 국민 혈세 42조 소요 2. 건보,

-------------------------------------------------
#91
Title :인면수심 문재인 개새끼!
Link :https://blog.naver.com/w76097698?Redirect=Log&logNo=221587947185
Description :#문재인 #촛불 #내란선동 언론을 이렇게 악용했습니다. 죄 하나 없은 박근혜 대통령님 #선의_무과실 로 문재인 개자식 선동질에 강금당하셨습니다. #동기부여강사 #자기계발강사 #인간승리자 #멘토 #정지만강사... 
Blogger Name :"동기부여 아카데미" 정지만강사 010-9195-0665
Blogger Link :https://blog.naver.com/w76097698
Post Date :20190717
-------------------------------------------------
#92
Title :문재인 공산주의자
Link :https://blog.naver.com/socoolo1?Redirect=Log&logNo=221562633946
Description :-北인민군의 뿌리 김원봉- 조선의용대의 진실 김원봉을 추켜세운 문재인 대통령의 현충일 추념사가 대한민국을 두 동강 냈다. 김원봉이 이끌던 조선의용대가 광복군에 참여한 것을 강조하면서 &quot;통합된 광복군은... 
Blogger Name :쏘쿨
Blogger Link :https://blog.naver.com/socoolo1
Post Date :20190615
-------------------------------------------------
#93
Title :[스마트그리드 관련주] 7월 4일 손정의 회장 문재인대통령 접견
Link :https://blog.naver.com/myplan76?Redirect=Log&logNo=221577115467
Description :&gt; 손정의 소프트뱅크 회장이 내일 문재인 대통령 접견에 이어 삼성 이재용 부회장, 현대차 정의선... 문재인 대통령과는 '

-------------------------------------------------
#121
Title :문재인과 청와대의 악마를 보았다 실사판
Link :http://soskr.egloos.com/1218784
Description :분노함 문재인은 사실상 악마를 초월한 사이코패스 수준 너는 스스로 정치안한다면서 구라칠때부터 알아봤다 신고 태그 : 문재인 , 김정은 , 북한 , 천안함 , 문재앙 , 민주당 , 청와대 , 이게나라다 포스트 메타... 
Blogger Name :골때리는 블로그
Blogger Link :http://soskr.egloos.com/
Post Date :20190608
-------------------------------------------------
#122
Title :문재인 외교는 구걸하듯!!
Link :http://blog.daum.net/softmanman/7087446
Description :대해선 문재인의 기억력처럼 전혀 예전일을 떠올리지 못 하지요. 그러면서 구걸하듯 징징대며 뮈텔... 얼마전 자유한국당 강효상 의원이 국회 정론관에서 문재인 대통령이 트럼프에게 일본에 방문하면 가까운... 
Blogger Name :자유게시판
Blogger Link :http://blog.daum.net/softmanman
Post Date :20190606
-------------------------------------------------
#123
Title :박원순 문재인 간첩?
Link :https://blog.naver.com/w76097698?Redirect=Log&logNo=221570437920
Description :kr/petitions/Temp/h3JYRl 해외여행 나가듯 거짓말 하는 문재인을 구속하자! &gt; 대한민국 청와대 나라를 나라답게, 국민과 함께 갑니다. www1.president.go.kr 정지만 사설 문재인이와 박원순이 공통점... 첫째... 
Blogger Name :"동기부여 아카데미

...

-------------------------------------------------
#861
Title :문재인 대통령의 아시아 뉴스 네트워크 기고문
Link :https://blog.naver.com/thebluehousekr?Redirect=Log&logNo=221482629028
Description :앞둔 문재인 대통령이 아시아 지역 언론 기고문을 통해 우리의 신남방정책과 아세안을 향한 비전을 밝혔습니다. 문재인 대통령은 &quot;아세안 친구들에게 대한민국 국민이 보내는 따뜻한 우정의 인사를 전한다... 
Blogger Name :대한민국 청와대 공식 블로그입니다.
Blogger Link :https://blog.naver.com/thebluehousekr
Post Date :20190307
-------------------------------------------------
#862
Title :문재인은 일구이언(一口二言)으로 평화를 말한다.
Link :http://blog.daum.net/msk1117/15974529
Description :문재인은 착각은 자유라지만 평화가 대화를 통해 이해로 이루어지는 것은 아니다 . 문재인이 노르웨이 오슬로대에서 열린 ‘ 오슬로 포럼 ’ 의 기조연설 중에서 그동안 듣도 보도 못한 평화론을 제기하였다... 
Blogger Name :김민상 참 고마워유
Blogger Link :http://blog.daum.net/msk1117
Post Date :20190614
-------------------------------------------------
#863
Title :&lt;윤창중칼럼&gt; 박근혜 대통령 끌어내리고 문재인을 앉힌 참혹한... 
Link :https://blog.naver.com/cjyoon214?Redirect=Log&logNo=221438130440
Description :https://youtu.be/7W4GyTHdl2U &lt;윤창중칼럼&gt; 박근혜 대통령 끌어내리고 문재인을

-------------------------------------------------
#891
Title :국채보상 동학운동 1세기 전으로 돌아간 듯한 문재인 대통령
Link :https://blog.naver.com/bbae46?Redirect=Log&logNo=221587681185
Description :지난주 문재인 대통령은 전남도청에서 &quot;전남 주민들은 이순신 장군과 함께 불과 열두 척 배로 나라를... 친일 청산을 강조한 문재인 대통령의 지난 3•1절 기념사는 원로 진보 학자로부터 &quot;관제(官製)... 
Blogger Name :bbae46님의 블로그
Blogger Link :https://blog.naver.com/bbae46
Post Date :20190717
-------------------------------------------------
#892
Title :문재인대통령 딸 다혜씨 해외이주 의혹 감사 않기로
Link :http://soskr.egloos.com/1218729
Description :감사원이 문재인 대통령의 딸 문다혜씨의 해외 이주 의혹에 대해서 공익감사를 하지 않기로... 그때 논란이 있었던 일부 민주당 의원들과 문재인 대통령에 대한 모든 의혹에 대해 전부 한번 까봅시다. 그저... 
Blogger Name :골때리는 블로그
Blogger Link :http://soskr.egloos.com/
Post Date :20190606
-------------------------------------------------
#893
Title :문재인 케어! 그 성과와 앞으로의 전망
Link :https://blog.naver.com/ktv520?Redirect=Log&logNo=221274984365
Description :2017년 8월 9일 문재인 대통령은 서울성모병원을 방문한 자리에서 향후 5년간 30조 6,000억 원을 들여... 수가 적용 대상 확대를 통한 건강보험 보장성 강화 정책으로 일명 '문재

-------------------------------------------------
#921
Title :청국장 잘 쳐먹대, 문재인 - 꼬마작가 정치시
Link :https://blog.naver.com/igor5474?Redirect=Log&logNo=221534577261
Description :쳐먹대, 문재인. 족보 있는 주사파 임종석을 비서실장으로 삼았고, 족보 있는 주사파 1기 이인영은 원내총무. 지난 2년을 돌아보니 주사파로 한가득. 안 먹어도 배부를 듯하오만, 청국장 잘 쳐먹대, 문재인.... 
Blogger Name :사샤와 이고리
Blogger Link :https://blog.naver.com/igor5474
Post Date :20190510
-------------------------------------------------
#922
Title :문재인 2년 그의 지지 세력들이 권력도 부도 독점 망국길로&lt;펌&gt;
Link :https://blog.naver.com/ykjdragon?Redirect=Log&logNo=221590526032
Description :문재인 2년 그의 지지 세력들이 권력도 부도 독점 망국길로 |이계성 칼럼아기천사|조회 14|추천 0|2019.07.20. 05:55http://cafe.daum.net/namjachansa/borg/1085 28:55 다음 동영상 재생지금 재생 문재인 6-1 사회교과서 저자... 
Blogger Name :ykjdragon님의블로그
Blogger Link :https://blog.naver.com/ykjdragon
Post Date :20190720
-------------------------------------------------
#923
Title :문재인이 불산공장 건설 반대?
Link :https://blog.naver.com/mobileoff?Redirect=Log&logNo=221581112233
Description :5년전에 문재인과 환경단체가 삼

-------------------------------------------------
#951
Title :오늘 문재인대통령님 일정
Link :https://blog.naver.com/sumi1056?Redirect=Log&logNo=221586324553
Description :#오늘 #문재인대통령님 #일정 #수소경제 #AI #자율주행자동차 #5G 등 에서 #협력 #강화 #약속 #작년 #한국 과 #이스라엘 #교역규모 #27억달러 알게 모르게 우리나라 이스라엘과 상당히 사이 좋습니다. #중동... 
Blogger Name :파격과 순수의 교차
Blogger Link :https://blog.naver.com/sumi1056
Post Date :20190715
-------------------------------------------------
#952
Title :문재인 대통령 통역관, 스펙보고 기절할뻔.
Link :https://blog.naver.com/treewg?Redirect=Log&logNo=221574375433
Description :문재인 대통령 통역관, 스펙보고 기절할뻔. 퍼온 글입니다) 문재인 대통령 통역관 스펙(사진속 인물 연세대 사회학과 01학번 채경훈 청와대행정관) 2007년 외무고시 합격(영어 능통자 전형) 영어 일어 중국어... 
Blogger Name :나무처럼
Blogger Link :https://blog.naver.com/treewg
Post Date :20190630
-------------------------------------------------
#953
Title :&lt;윤창중칼럼&gt; ‘태양광 문재인’ 미쳤다
Link :https://blog.naver.com/cjyoon214?Redirect=Log&logNo=221388047248
Description :문재인’ 미쳤다 세계 최장이자 서울 면적의 3분2에 해당하는, 새만금 입구에서부터 입이 다물어지지 않게 하는 그 드넓은 방조제를 태양광 패널로 깔

-------------------------------------------------
#981
Title :'문재인 탄핵 거론' 후지TV 20190718 진성호 外
Link :http://blog.daum.net/soonchang4623/3789
Description :문재인 탄핵 거론하다 . 진성호의 직설) 진성호방송 게시일: 2019. 7. 18. [정참시] 日 TV, &quot; 문재인 탄핵 여야 총선에 ' 日 폭탄 (2019.07.18/뉴스데스크/MBC) 게시일: 2019. 7. 18. 첫 번째 소식 볼까요? 문재인 탄핵, 일본... 
Blogger Name :하늘나라 -2-
Blogger Link :http://blog.daum.net/soonchang4623
Post Date :20190718
-------------------------------------------------
#982
Title :문재인이라는 인물은 현 시대에 적합한 인물은 아니다.
Link :https://blog.naver.com/orion_203?Redirect=Log&logNo=221500871023
Description :노무현 대통령과 문재인 대통령으로 이어지는 민주진영에 핵심, 아니 더나가 김대중 대통령까지..... 문재인 정부에서 터진 김학의 재조사를 위한 특별수사단장에 여환섭 검사를 임명한것에 대해서 검찰에... 
Blogger Name :저열한 인간들의 기록.
Blogger Link :https://blog.naver.com/orion_203
Post Date :20190330
-------------------------------------------------
#983
Title :예고 – 문재인 정부 2년 특집 대담 ‘대통령에게 묻는다’
Link :https://blog.naver.com/thebluehousekr?Redirect=Log&logNo=221533351267
Description :취임 2주년을 하루 앞둔 오늘 저녁, 문재인 대통령이 국